## 一些需要明确的知识

1.Python中一切皆对象(`object`),函数也是一个对象(`code object`).

函数是一个可调用(`callable`)的对象.

In [1]:
def f():
    pass
f

<function __main__.f()>

2.Python中函数可以作为参数传入另一个函数

In [2]:
def double(x):
    return x * 2

def triple(x):
    return x * 3

# 函数作为一个参数传入另一个函数
def calc_number(func, x):
    print(func(x))

calc_number(double, 3)
calc_number(triple, 3)

6
9


3.Python中函数的返回值也可以是一个函数。

In [3]:
def get_multiple_func(n):

    def multiple(x):
        return n * x

    return multiple

double = get_multiple_func(2)
triple = get_multiple_func(3)

print(double(3))
print(triple(3))

6
9


## Decorator详解

### 函数类型的Decorator

In [4]:
def dec(f):
    pass

@dec
def double(x):
    return x * 2

# # 完全等价于：
# double = dec(double)

上述代码完全等价于`double = dec(double)`,
可以看出装饰器`@dec`本质上是一个输入输出都为函数的函数(输出不一定强制要求为函数，但是一般使用输出都为函数)。

下面给出一个具体的例子：

In [5]:
import time

# 装饰器，打印函数每次允许的时间
def timeit(f):
    # 输入为一个函数 
    def wrapper(x):
        start = time.time()
        ret = f(x)
        print(time.time() - start)
        return ret
    # 返回也是一个函数
    return wrapper

@timeit
def my_func(x):
    time.sleep(x)

@timeit
def other_func(x):
    return x * 2

print(my_func(1))
print(other_func(1))

1.0010404586791992
None
1.9073486328125e-06
2


标准写法：使用变长函数参数

In [6]:
import time

# 装饰器，打印函数每次允许的时间
def timeit(f):
    # 输入为一个函数 
    def wrapper(*args, **kwargs):
        # 函数接收变长参数
        start = time.time()
        ret = f(*args, **kwargs)
        print(time.time() - start)
        return ret
    # 返回也是一个函数
    return wrapper

@timeit
def my_func(x):
    time.sleep(x)

@timeit
def add(x, y):
    return x + y

print(my_func(1))
print(add(1, 2))

1.0010452270507812
None
3.337860107421875e-06
3


进阶：带参数的装饰器

In [7]:
import time

def timeit(iteration):
    # 打印运行iteration次的运行时间
    def inner(f):
        def wrapper(*args, **kwargs):
            start = time.time()
            for _ in range(iteration):
                ret = f(*args, **kwargs)
            print(time.time() - start)
            return ret
        return wrapper
    return inner

@timeit(3)
def my_func(x):
    time.sleep(x)

@timeit(1000)
def add(x, y):
    return x + y

print(my_func(1))
print(add(1, 2))

3.003140926361084
None
0.00033164024353027344
3


上述代码等价于:
```python
inner = timeit(1000)
add = inner(add)
```

### Class类型的装饰器(装饰器类)

In [8]:
import time

class Timer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        start = time.time()
        ret = self.func(*args, **kwargs)
        print(f"Time:{time.time()-start}")
        return ret

@Timer
def add(a, b):
    return a + b

print(type(add))
print(add(1, 3))

<class '__main__.Timer'>
Time:9.5367431640625e-07
4


上述代码等价于：`add = Timer(add)`

下面展示带参数的例子：

In [9]:
class Timer:
    def __init__(self, prefix):
        self.prefix = prefix

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            start = time.time()
            ret = func(*args, **kwargs)
            print(f"{self.prefix}{time.time() - start}")
            return ret
        return wrapper

@Timer(prefix="curr_time:")
def add(a, b):
    return a + b

print(add(1, 3))

curr_time:9.5367431640625e-07
4


上述代码等价于：`add = Timer(prefix='current_time:')(add)`

### 装饰类的装饰器(类装饰器)

In [10]:
def add_str(cls):
    def __str__(self):
        return str(self.__dict__)
    
    cls.__str__ = __str__
    return cls

@add_str
class MyObject:
    def __init__(self, a, b):
        self.a = a
        self.b = b

o = MyObject(1, 2)
print(o)

{'a': 1, 'b': 2}


上述代码等价于：`MyObject = add_str(MyObject)`